除了它自己那个不知道怎么得到的公式以外，线性回归的baseline也是重要的。也要分别试试log化的和没有log化的

In [1]:
import pandas as pd

import statsmodels.formula.api as smf
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
df = pd.read_excel('GFR-SVR数据.xls')
df.columns = ['id', 'age', 'sex', 'rGFR', 'Scr', 'Cys']
df['Cys'] = pd.to_numeric(df['Cys'], errors='coerce')
df=df.dropna()
df.shape

(1188, 6)

In [3]:
from sklearn.linear_model import LinearRegression

In [6]:
dfp = df.loc[:df.shape[0]-1] # last record is hard to deal

X = df[['age', 'Scr', 'Cys']]
y = df['rGFR']

X_sex = df[['age','Scr','Cys','sex']]

yp = np.log(dfp['rGFR'])
Xp = np.stack([dfp['age'], np.log(dfp['Scr']), np.log(dfp['Cys'])], 1)

yp = np.log(dfp['rGFR'])
Xp_sex = np.stack([dfp['age'], np.log(dfp['Scr']), np.log(dfp['Cys']), dfp['sex']], 1)

In [13]:
model = LinearRegression()
y_pred = model.fit(X,y).predict(X)
np.mean((y - y_pred)**2)

317.6063130597567

In [14]:
model_sex = LinearRegression()
y_sex_pred = model_sex.fit(X_sex,y).predict(X_sex)
np.mean((y - y_sex_pred)**2)

314.4968821852696

In [15]:
model_log = LinearRegression()
yp_pred = model_log.fit(Xp,yp).predict(Xp)
np.mean((np.exp(yp_pred) - np.exp(yp))**2)

234.59113039423264

In [16]:
model_log_sex = LinearRegression()
yp_sex_pred = model_log_sex.fit(Xp_sex,yp).predict(Xp_sex)
np.mean((np.exp(yp_sex_pred) - np.exp(yp))**2)

235.92777092849204

In [17]:
model_log_sex.coef_

array([-0.00346906, -0.36474529, -0.44646327,  0.03449203])

In [20]:
model_log_sex.intercept_

4.445729428272579

In [21]:
np.exp(_)

85.26204770851174

In [23]:
np.log(0.993)

-0.007024614936964466

和经验公式的系数类似

不过这样对比也不太公平，线性回归用上了所有样本，效果自然要好一点，虽然它倒也不可能过拟合，但为了公平起见还是改成0.2比例样本分离出去。

In [24]:
from sklearn.model_selection import train_test_split


In [25]:
Xp_train, Xp_test, yp_train, yp_test = train_test_split(Xp_sex, yp)

In [26]:
model_log_sex = LinearRegression()
model_log_sex.fit(Xp_train,yp_train)
yp_train_pred = model_log_sex.predict(Xp_train)
yp_test_pred = model_log_sex.predict(Xp_test)

np.mean((np.exp(yp_train_pred) - np.exp(yp_train))**2),np.mean((np.exp(yp_test_pred) - np.exp(yp_test))**2)

(228.4732652710458, 268.47057711248624)

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
model_sex = LinearRegression()
model_sex.fit(Xp_train,yp_train)
yp_train_pred = model_log_sex.predict(Xp_train)
yp_test_pred = model_log_sex.predict(Xp_test)
